# TFM

## Importación de librerías

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import toml
import random
import tensorflow as tf
from sklearn.metrics import (
    accuracy_score, 
    f1_score,
    recall_score,
    precision_score,
    confusion_matrix,
    ConfusionMatrixDisplay
)

## Ejecución de código en GPU

In [24]:
import tensorflow as tf

# Comprobar la versión de TensorFlow
print("TensorFlow version:", tf.__version__)

# Listar todos los dispositivos físicos disponibles
physical_devices = tf.config.list_physical_devices('GPU')

if physical_devices:
    print("🟢 TensorFlow ha detectado la(s) siguiente(s) GPU(s):")
    for gpu in physical_devices:
        print(f'\t{gpu}')
else:
    print("🔴 No se ha detectado ninguna GPU. TensorFlow está utilizando la CPU.")

TensorFlow version: 2.10.0
🟢 TensorFlow ha detectado la(s) siguiente(s) GPU(s):
	PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


## Lectura de fichero

In [25]:
data_dict = {}
DATA_DIR = '../../data/'

for data_folder in os.listdir(DATA_DIR):
    *data_files, config_file = os.listdir(f'{DATA_DIR}/{data_folder}')

    leak_value = toml.load(f'{DATA_DIR}/{data_folder}/{config_file}')['tanks']['flow_value']

    train_dataframe, test_data = random.sample(data_files, 2)

    data_dict[data_folder] = {
        'leak_value': leak_value,
        'train_dataframe': pd.read_csv(f'{DATA_DIR}/{data_folder}/{train_dataframe}').drop(columns=['Unnamed: 0']),
        'test_dataframe': pd.read_csv(f'{DATA_DIR}/{data_folder}/{test_data}').drop(columns=['Unnamed: 0'])
    }

### Conjuntos de datos de entrenamiento

In [26]:
for case in data_dict:
    display(data_dict[case]['train_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"21950,34",-6035,"7006,85","22922,19","22962,99","40,8","172,2",0
1,2,"22962,99",-6105,"6515,76","23373,75","23300,26","-73,49","125,79",0
2,3,"23300,26",-5770,"6438,69","23968,95","24020,7","51,75","97,38",0
3,4,"24020,7",-5915,"5427,88","23533,58","23536,74","3,16","79,51",0
4,5,"23536,74",-6415,"4351,1","21472,84","21473,49","0,65","87,66",0
...,...,...,...,...,...,...,...,...,...
995,996,"15927,66",-6095,"6450,17","16282,83","16239,24","-43,59","114,14",1
996,997,"16239,24",-5855,"6019,11","16403,35","16385,84","-17,51","38,4",1
997,998,"16385,84",-6120,"3750,83","14016,67","13979,49","-37,18","-21,93",1
998,999,"13979,49",-5845,"6725,82","14860,31","14857,03","-3,28","10,77",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"12858,53","-6005,0","6028,71","12882,24","12839,43","-42,81","67,14",0
1,2,"12839,43","-5925,0","7060,68","13975,11","13920,05","-55,06","3,95",0
2,3,"13920,05","-6450,0","11910,17","19380,22","19446,63","66,41","-28,04",0
3,4,"19446,63","-6495,0","7059,14","20010,77","19951,08","-59,69","-92,41",0
4,5,"19951,08","-6300,0","7021,46","20672,54","20708,94","36,4","-130,33",0
...,...,...,...,...,...,...,...,...,...
995,996,"3854,5","-5960,0","5534,05","3428,55","3355,21","-73,34","-305,54",1
996,997,"3355,21","-6370,0","6329,75","3314,96","3330,31","15,35","-300,52",1
997,998,"3330,31","-5760,0","2905,72","476,03","483,45","7,42","-273,07",1
998,999,"483,45","-5412,11","8221,51","3292,85","3340,43","47,58","-195,76",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"20423,93","-5590,0","5813,6","20647,53","20665,41","17,88","31,64",0
1,2,"20665,41","-5970,0","5997,78","20693,19","20680,86","-12,33","79,51",0
2,3,"20680,86","-5930,0","7951,68","22702,54","22757,13","54,59","150,95",0
3,4,"22757,13","-6260,0","6680,1","23177,23","23197,64","20,41","107,33",0
4,5,"23197,64","-6075,0","5816,09","22938,73","22983,34","44,61","181,99",0
...,...,...,...,...,...,...,...,...,...
995,996,"2,36","-0,0","0,0","2,36","14,55","12,19","2,84",1
996,997,"14,55","-0,0","0,0","14,55","4,86","-9,69","-1,12",1
997,998,"4,86","-0,0","0,0","4,86","0,46","-4,4","0,26",1
998,999,"0,46","-0,0","0,0","0,46","11,22","10,76","-9,79",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"19850,01","-6190,0","3007,83","16667,84","16649,01","-18,83","101,86",0
1,2,"16649,01","-6075,0","7996,41","18570,42","18569,16","-1,26","38,55",0
2,3,"18569,16","-6040,0","7973,61","20502,77","20544,29","41,52","126,92",0
3,4,"20544,29","-6335,0","5734,95","19944,24","19952,92","8,68","69,48",0
4,5,"19952,92","-6275,0","6590,42","20268,34","20302,97","34,63","41,44",0
...,...,...,...,...,...,...,...,...,...
995,996,"1,57","-0,0","0,0","1,57","3,03","1,46","2,57",1
996,997,"3,03","-0,0","0,0","3,03","1,28","-1,75","-6,25",1
997,998,"1,28","-0,0","0,0","1,28","16,88","15,6","5,54",1
998,999,"16,88","-0,0","0,0","16,88","2,92","-13,96","-7,19",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"20535,61","-5940,0","8641,12","23236,73","23294,75","58,02","265,22",0
1,2,"23294,75","-6575,0","6455,26","23175,01","23296,05","121,04","387,15",0
2,3,"23296,05","-5630,0","6600,92","24266,97","24325,7","58,73","366,14",0
3,4,"24325,7","-6015,0","3190,31","21501,01","21523,46","22,45","375,09",0
4,5,"21523,46","-5695,0","7376,83","23205,29","23238,07","32,78","376,68",0
...,...,...,...,...,...,...,...,...,...
995,996,"3,69","-0,0","0,0","3,69","23,8","20,11","18,39",1
996,997,"23,8","-0,0","0,0","23,8","15,45","-8,35","12,43",1
997,998,"15,45","-0,0","0,0","15,45","10,85","-4,6","-0,6",1
998,999,"10,85","-0,0","0,0","10,85","3,49","-7,36","-0,14",1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22143,65","-6055,0","6215,36","22304,01","22242,32","-61,69","34,11",0
1,2,"22242,32","-5940,0","6691,65","22993,97","22983,66","-10,31","42,41",0
2,3,"22983,66","-5885,0","6130,98","23229,64","23301,88","72,24","58,48",0
3,4,"23301,88","-6030,0","5035,77","22307,65","22270,88","-36,77","25,47",0
4,5,"22270,88","-6005,0","6518,15","22784,03","22794,42","10,39","13,96",0
...,...,...,...,...,...,...,...,...,...
995,996,"2,74","-0,0","0,0","2,74","3,88","1,14","3,0",1
996,997,"3,88","-0,0","0,0","3,88","0,72","-3,16","-2,72",1
997,998,"0,72","-0,0","0,0","0,72","1,13","0,41","-11,02",1
998,999,"1,13","-0,0","0,0","1,13","2,14","1,01","-19,34",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"15886,17","-5900,0","5970,3","15956,47","15980,01","23,54","68,46",0
1,2,"15980,01","-5810,0","6050,74","16220,75","16175,65","-45,1","8,64",0
2,3,"16175,65","-5760,0","5917,51","16333,16","16413,96","80,8","17,3",0
3,4,"16413,96","-5970,0","6982,8","17426,76","17460,02","33,26","68,23",0
4,5,"17460,02","-6340,0","7929,64","19049,66","19111,41","61,75","52,84",0
...,...,...,...,...,...,...,...,...,...
995,996,"5,52","-0,0","0,0","5,52","0,47","-5,05","-4,84",1
996,997,"0,47","-0,0","0,0","0,47","5,79","5,32","0,68",1
997,998,"5,79","-0,0","0,0","5,79","14,14","8,35","-0,25",1
998,999,"14,14","-0,0","0,0","14,14","4,73","-9,41","2,66",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"19999,39","-5905,0","6970,79","21065,18","21102,9","37,72","109,13",0
1,2,"21102,9","-6155,0","6060,28","21008,18","20966,69","-41,49","59,33",0
2,3,"20966,69","-5975,0","2934,69","17926,38","17990,19","63,81","85,1",0
3,4,"17990,19","-6305,0","6950,94","18636,13","18679,74","43,61","127,56",0
4,5,"18679,74","-5940,0","7943,2","20682,94","20731,84","48,9","178,79",0
...,...,...,...,...,...,...,...,...,...
995,996,"5,73","-0,0","0,0","5,73","3,13","-2,6","-19,24",1
996,997,"3,13","-0,0","0,0","3,13","3,87","0,74","-3,1",1
997,998,"3,87","-0,0","0,0","3,87","5,88","2,01","5,53",1
998,999,"5,88","-0,0","0,0","5,88","2,22","-3,66","-1,13",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"13606,56","-6135,0","6078,24","13549,8","13604,57","54,77","189,82",0
1,2,"13604,57","-6180,0","7816,29","15240,86","15298,64","57,78","140,18",0
2,3,"15298,64","-6540,0","8066,41","16825,05","16766,16","-58,89","114,81",0
3,4,"16766,16","-5950,0","2994,85","13811,01","13832,37","21,36","118,61",0
4,5,"13832,37","-6225,0","8046,4","15653,77","15600,11","-53,66","107,3",0
...,...,...,...,...,...,...,...,...,...
995,996,"3,77","-0,0","0,0","3,77","5,54","1,77","-2,21",1
996,997,"5,54","-0,0","0,0","5,54","0,83","-4,71","-9,32",1
997,998,"0,83","-0,0","0,0","0,83","13,25","12,42","1,59",1
998,999,"13,25","-0,0","0,0","13,25","10,31","-2,94","6,54",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22883,35","-6050,0","5453,86","22287,21","22230,52","-56,69","249,29",0
1,2,"22230,52","-6275,0","3883,29","19838,81","19981,1","142,29","345,12",0
2,3,"19981,1","-6150,0","7525,49","21356,59","21346,06","-10,53","273,17",0
3,4,"21346,06","-6130,0","6948,29","22164,35","22227,3","62,95","280,51",0
4,5,"22227,3","-5815,0","5639,5","22051,8","22112,87","61,07","286,84",0
...,...,...,...,...,...,...,...,...,...
995,996,"1,47","-0,0","0,0","1,47","15,25","13,78","10,81",1
996,997,"15,25","-0,0","0,0","15,25","6,77","-8,48","-2,84",1
997,998,"6,77","-0,0","0,0","6,77","1,04","-5,73","-11,61",1
998,999,"1,04","-0,0","0,0","1,04","22,11","21,07","14,93",1


### Conjunto de datos de prueba

In [27]:
for case in data_dict:
    display(data_dict[case]['test_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"14217,87",-6025,"8078,98","16271,85","16202,83","-69,02","29,76",0
1,2,"16202,83",-5915,"7993,5","18281,33","18287,13","5,8","103,63",0
2,3,"18287,13",-6005,"5997,67","18279,8","18294,11","14,31","27,16",0
3,4,"18294,11",-6085,"3013,11","15222,22","15202,29","-19,93","93,76",0
4,5,"15202,29",-5800,"7941,86","17344,15","17431,85","87,7","159,23",0
...,...,...,...,...,...,...,...,...,...
995,996,"15406,65",-6030,"4978,48","14355,13","14337,52","-17,61","61,26",1
996,997,"14337,52",-6000,"5721,51","14059,03","14009,35","-49,68","-44,29",1
997,998,"14009,35",-6275,"6313,19","14047,54","14181,99","134,45","161,32",1
998,999,"14181,99",-6150,"5914,01","13946,0","14007,4","61,4","181,88",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"17135,63","-5975,0","7990,02","19150,65","19183,49","32,84","106,49",0
1,2,"19183,49","-6310,0","7021,06","19894,55","19852,62","-41,93","88,07",0
2,3,"19852,62","-5855,0","7068,29","21065,91","21015,88","-50,03","36,94",0
3,4,"21015,88","-5995,0","5991,07","21011,95","21025,1","13,15","-10,55",0
4,5,"21025,1","-6045,0","6914,83","21894,93","21927,46","32,53","-67,99",0
...,...,...,...,...,...,...,...,...,...
995,996,"2535,76","-6170,0","6732,66","3098,42","3084,18","-14,24","-244,63",1
996,997,"3084,18","-6050,0","5225,99","2260,17","2291,41","31,24","-67,43",1
997,998,"2291,41","-5735,0","7144,7","3701,11","3650,86","-50,25","-129,83",1
998,999,"3650,86","-5915,0","3212,68","948,54","914,6","-33,94","-248,7",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"23654,87","-6005,0","4298,93","21948,8","21972,61","23,81","163,31",0
1,2,"21972,61","-6095,0","4622,96","20500,57","20617,61","117,04","322,76",0
2,3,"20617,61","-6175,0","9067,79","23510,4","23442,51","-67,89","221,84",0
3,4,"23442,51","-5990,0","3410,32","20862,83","20890,7","27,87","233,22",0
4,5,"20890,7","-5915,0","6810,38","21786,08","21803,84","17,76","215,53",0
...,...,...,...,...,...,...,...,...,...
995,996,"10,65","-0,0","0,0","10,65","30,77","20,12","26,54",1
996,997,"30,77","-0,0","0,0","30,77","0,47","-30,3","-3,4",1
997,998,"0,47","-0,0","0,0","0,47","4,08","3,61","0,87",1
998,999,"4,08","-0,0","0,0","4,08","1,82","-2,26","-3,71",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"17445,18","-5905,0","6012,94","17553,12","17533,44","-19,68","16,06",0
1,2,"17533,44","-6110,0","8506,7","19930,14","20021,08","90,94","102,33",0
2,3,"20021,08","-5930,0","4407,28","18498,36","18474,61","-23,75","162,19",0
3,4,"18474,61","-6080,0","8762,8","21157,41","21038,06","-119,35","-12,14",0
4,5,"21038,06","-6210,0","8428,75","23256,81","23192,92","-63,89","-100,6",0
...,...,...,...,...,...,...,...,...,...
995,996,"8,86","-0,0","0,0","8,86","9,4","0,54","3,9",1
996,997,"9,4","-0,0","0,0","9,4","12,79","3,39","2,6",1
997,998,"12,79","-0,0","0,0","12,79","14,69","1,9","-0,82",1
998,999,"14,69","-0,0","0,0","14,69","9,11","-5,58","-12,74",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"17548,09","-6265,0","4958,31","16241,4","16245,87","4,47","160,94",0
1,2,"16245,87","-5725,0","6084,29","16605,16","16550,86","-54,3","43,48",0
2,3,"16550,86","-5880,0","6957,27","17628,13","17694,1","65,97","93,95",0
3,4,"17694,1","-6025,0","7912,37","19581,47","19651,53","70,06","229,9",0
4,5,"19651,53","-5880,0","5996,13","19767,66","19771,94","4,28","319,27",0
...,...,...,...,...,...,...,...,...,...
995,996,"9,49","-0,0","0,0","9,49","2,52","-6,97","-20,78",1
996,997,"2,52","-0,0","0,0","2,52","1,6","-0,92","-5,93",1
997,998,"1,6","-0,0","0,0","1,6","7,92","6,32","-9,84",1
998,999,"7,92","-0,0","0,0","7,92","1,37","-6,55","-12,18",1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"16942,12","-5825,0","5491,42","16608,54","16673,87","65,33","187,29",0
1,2,"16673,87","-5825,0","6897,1","17745,97","17875,86","129,89","237,48",0
2,3,"17875,86","-5965,0","6950,25","18861,11","18894,66","33,55","253,19",0
3,4,"18894,66","-5970,0","7962,56","20887,22","20937,92","50,7","387,03",0
4,5,"20937,92","-6135,0","5924,27","20727,19","20818,82","91,63","502,42",0
...,...,...,...,...,...,...,...,...,...
995,996,"19,68","-0,0","0,0","19,68","8,25","-11,43","0,72",1
996,997,"8,25","-0,0","0,0","8,25","3,65","-4,6","-8,36",1
997,998,"3,65","-0,0","0,0","3,65","8,4","4,75","4,92",1
998,999,"8,4","-0,0","0,0","8,4","23,49","15,09","16,35",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"18109,75","-5940,0","6999,66","19169,41","19140,47","-28,94","290,5",0
1,2,"19140,47","-6320,0","7014,09","19834,56","19825,75","-8,81","346,46",0
2,3,"19825,75","-6145,0","7026,85","20707,6","20677,99","-29,61","326,59",0
3,4,"20677,99","-5805,0","4783,79","19656,78","19702,96","46,18","340,89",0
4,5,"19702,96","-6195,0","6592,28","20100,24","20021,69","-78,55","117,34",0
...,...,...,...,...,...,...,...,...,...
995,996,"3,39","-0,0","0,0","3,39","8,94","5,55","-9,07",1
996,997,"8,94","-0,0","0,0","8,94","13,46","4,52","1,87",1
997,998,"13,46","-0,0","0,0","13,46","5,2","-8,26","4,92",1
998,999,"5,2","-0,0","0,0","5,2","13,18","7,98","-1,52",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22465,51","-6080,0","7789,55","24175,06","24208,87","33,81","-175,32",0
1,2,"24208,87","-6045,0","4948,93","23112,8","23139,14","26,34","-178,39",0
2,3,"23139,14","-5935,0","7036,17","24240,31","24285,57","45,26","-129,43",0
3,4,"24285,57","-5950,0","4812,44","23148,01","23215,86","67,85","74,7",0
4,5,"23215,86","-6310,0","5513,33","22419,19","22370,85","-48,34","67,2",0
...,...,...,...,...,...,...,...,...,...
995,996,"13,88","-0,0","0,0","13,88","5,95","-7,93","1,78",1
996,997,"5,95","-0,0","0,0","5,95","12,1","6,15","10,02",1
997,998,"12,1","-0,0","0,0","12,1","5,06","-7,04","-9,98",1
998,999,"5,06","-0,0","0,0","5,06","4,17","-0,89","-3,22",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"17698,38","-5710,0","4618,64","16607,02","16580,51","-26,51","35,66",0
1,2,"16580,51","-5875,0","6004,3","16709,81","16702,31","-7,5","58,61",0
2,3,"16702,31","-6250,0","8001,28","18453,59","18456,14","2,55","-9,07",0
3,4,"18456,14","-5870,0","6931,83","19517,97","19595,08","77,11","-16,24",0
4,5,"19595,08","-6330,0","7449,42","20714,5","20644,03","-70,47","-108,89",0
...,...,...,...,...,...,...,...,...,...
995,996,"1,11","-0,0","0,0","1,11","1,53","0,42","-12,95",1
996,997,"1,53","-0,0","0,0","1,53","5,18","3,65","0,97",1
997,998,"5,18","-0,0","0,0","5,18","9,61","4,43","9,4",1
998,999,"9,61","-0,0","0,0","9,61","13,39","3,78","3,06",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22379,58","-5785,0","5690,39","22284,97","22319,5","34,53","144,59",0
1,2,"22319,5","-5980,0","6499,32","22838,82","22799,39","-39,43","171,58",0
2,3,"22799,39","-6000,0","6305,86","23105,25","23145,46","40,21","213,48",0
3,4,"23145,46","-6095,0","5814,71","22865,17","22845,93","-19,24","195,21",0
4,5,"22845,93","-6145,0","6270,76","22971,69","22978,1","6,41","290,05",0
...,...,...,...,...,...,...,...,...,...
995,996,"2,71","-0,0","0,0","2,71","2,61","-0,1","-0,48",1
996,997,"2,61","-0,0","0,0","2,61","5,93","3,32","1,85",1
997,998,"5,93","-0,0","0,0","5,93","13,8","7,87","5,27",1
998,999,"13,8","-0,0","0,0","13,8","6,1","-7,7","-0,73",1


## Limpieza de datos

In [28]:
bad_formatted_fields = [
    'Volumen dep. almacenam. ini. (L)',
    'Llenado dep. almacenam. (L)',
    'Volumen dep. almacenam. fin. teor. (L)',
    'Volumen dep. almacenam. fin. (L)',
    'Variacion',
    'Variacion Acum.',
]

for case in data_dict:
    for dataset in ['train_dataframe', 'test_dataframe']:
        for field in bad_formatted_fields:
            data_dict[case][dataset][field] = data_dict[case][dataset][field].str.replace(',', '.')
            data_dict[case][dataset][field] = pd.to_numeric(data_dict[case][dataset][field])

### Conjunto de datos de entrenamiento

In [29]:
for case in data_dict:
    display(data_dict[case]['train_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,21950.34,-6035,7006.85,22922.19,22962.99,40.80,172.20,0
1,2,22962.99,-6105,6515.76,23373.75,23300.26,-73.49,125.79,0
2,3,23300.26,-5770,6438.69,23968.95,24020.70,51.75,97.38,0
3,4,24020.70,-5915,5427.88,23533.58,23536.74,3.16,79.51,0
4,5,23536.74,-6415,4351.10,21472.84,21473.49,0.65,87.66,0
...,...,...,...,...,...,...,...,...,...
995,996,15927.66,-6095,6450.17,16282.83,16239.24,-43.59,114.14,1
996,997,16239.24,-5855,6019.11,16403.35,16385.84,-17.51,38.40,1
997,998,16385.84,-6120,3750.83,14016.67,13979.49,-37.18,-21.93,1
998,999,13979.49,-5845,6725.82,14860.31,14857.03,-3.28,10.77,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,12858.53,"-6005,0",6028.71,12882.24,12839.43,-42.81,67.14,0
1,2,12839.43,"-5925,0",7060.68,13975.11,13920.05,-55.06,3.95,0
2,3,13920.05,"-6450,0",11910.17,19380.22,19446.63,66.41,-28.04,0
3,4,19446.63,"-6495,0",7059.14,20010.77,19951.08,-59.69,-92.41,0
4,5,19951.08,"-6300,0",7021.46,20672.54,20708.94,36.40,-130.33,0
...,...,...,...,...,...,...,...,...,...
995,996,3854.50,"-5960,0",5534.05,3428.55,3355.21,-73.34,-305.54,1
996,997,3355.21,"-6370,0",6329.75,3314.96,3330.31,15.35,-300.52,1
997,998,3330.31,"-5760,0",2905.72,476.03,483.45,7.42,-273.07,1
998,999,483.45,"-5412,11",8221.51,3292.85,3340.43,47.58,-195.76,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,20423.93,"-5590,0",5813.60,20647.53,20665.41,17.88,31.64,0
1,2,20665.41,"-5970,0",5997.78,20693.19,20680.86,-12.33,79.51,0
2,3,20680.86,"-5930,0",7951.68,22702.54,22757.13,54.59,150.95,0
3,4,22757.13,"-6260,0",6680.10,23177.23,23197.64,20.41,107.33,0
4,5,23197.64,"-6075,0",5816.09,22938.73,22983.34,44.61,181.99,0
...,...,...,...,...,...,...,...,...,...
995,996,2.36,"-0,0",0.00,2.36,14.55,12.19,2.84,1
996,997,14.55,"-0,0",0.00,14.55,4.86,-9.69,-1.12,1
997,998,4.86,"-0,0",0.00,4.86,0.46,-4.40,0.26,1
998,999,0.46,"-0,0",0.00,0.46,11.22,10.76,-9.79,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,19850.01,"-6190,0",3007.83,16667.84,16649.01,-18.83,101.86,0
1,2,16649.01,"-6075,0",7996.41,18570.42,18569.16,-1.26,38.55,0
2,3,18569.16,"-6040,0",7973.61,20502.77,20544.29,41.52,126.92,0
3,4,20544.29,"-6335,0",5734.95,19944.24,19952.92,8.68,69.48,0
4,5,19952.92,"-6275,0",6590.42,20268.34,20302.97,34.63,41.44,0
...,...,...,...,...,...,...,...,...,...
995,996,1.57,"-0,0",0.00,1.57,3.03,1.46,2.57,1
996,997,3.03,"-0,0",0.00,3.03,1.28,-1.75,-6.25,1
997,998,1.28,"-0,0",0.00,1.28,16.88,15.60,5.54,1
998,999,16.88,"-0,0",0.00,16.88,2.92,-13.96,-7.19,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,20535.61,"-5940,0",8641.12,23236.73,23294.75,58.02,265.22,0
1,2,23294.75,"-6575,0",6455.26,23175.01,23296.05,121.04,387.15,0
2,3,23296.05,"-5630,0",6600.92,24266.97,24325.70,58.73,366.14,0
3,4,24325.70,"-6015,0",3190.31,21501.01,21523.46,22.45,375.09,0
4,5,21523.46,"-5695,0",7376.83,23205.29,23238.07,32.78,376.68,0
...,...,...,...,...,...,...,...,...,...
995,996,3.69,"-0,0",0.00,3.69,23.80,20.11,18.39,1
996,997,23.80,"-0,0",0.00,23.80,15.45,-8.35,12.43,1
997,998,15.45,"-0,0",0.00,15.45,10.85,-4.60,-0.60,1
998,999,10.85,"-0,0",0.00,10.85,3.49,-7.36,-0.14,1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22143.65,"-6055,0",6215.36,22304.01,22242.32,-61.69,34.11,0
1,2,22242.32,"-5940,0",6691.65,22993.97,22983.66,-10.31,42.41,0
2,3,22983.66,"-5885,0",6130.98,23229.64,23301.88,72.24,58.48,0
3,4,23301.88,"-6030,0",5035.77,22307.65,22270.88,-36.77,25.47,0
4,5,22270.88,"-6005,0",6518.15,22784.03,22794.42,10.39,13.96,0
...,...,...,...,...,...,...,...,...,...
995,996,2.74,"-0,0",0.00,2.74,3.88,1.14,3.00,1
996,997,3.88,"-0,0",0.00,3.88,0.72,-3.16,-2.72,1
997,998,0.72,"-0,0",0.00,0.72,1.13,0.41,-11.02,1
998,999,1.13,"-0,0",0.00,1.13,2.14,1.01,-19.34,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,15886.17,"-5900,0",5970.30,15956.47,15980.01,23.54,68.46,0
1,2,15980.01,"-5810,0",6050.74,16220.75,16175.65,-45.10,8.64,0
2,3,16175.65,"-5760,0",5917.51,16333.16,16413.96,80.80,17.30,0
3,4,16413.96,"-5970,0",6982.80,17426.76,17460.02,33.26,68.23,0
4,5,17460.02,"-6340,0",7929.64,19049.66,19111.41,61.75,52.84,0
...,...,...,...,...,...,...,...,...,...
995,996,5.52,"-0,0",0.00,5.52,0.47,-5.05,-4.84,1
996,997,0.47,"-0,0",0.00,0.47,5.79,5.32,0.68,1
997,998,5.79,"-0,0",0.00,5.79,14.14,8.35,-0.25,1
998,999,14.14,"-0,0",0.00,14.14,4.73,-9.41,2.66,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,19999.39,"-5905,0",6970.79,21065.18,21102.90,37.72,109.13,0
1,2,21102.90,"-6155,0",6060.28,21008.18,20966.69,-41.49,59.33,0
2,3,20966.69,"-5975,0",2934.69,17926.38,17990.19,63.81,85.10,0
3,4,17990.19,"-6305,0",6950.94,18636.13,18679.74,43.61,127.56,0
4,5,18679.74,"-5940,0",7943.20,20682.94,20731.84,48.90,178.79,0
...,...,...,...,...,...,...,...,...,...
995,996,5.73,"-0,0",0.00,5.73,3.13,-2.60,-19.24,1
996,997,3.13,"-0,0",0.00,3.13,3.87,0.74,-3.10,1
997,998,3.87,"-0,0",0.00,3.87,5.88,2.01,5.53,1
998,999,5.88,"-0,0",0.00,5.88,2.22,-3.66,-1.13,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,13606.56,"-6135,0",6078.24,13549.80,13604.57,54.77,189.82,0
1,2,13604.57,"-6180,0",7816.29,15240.86,15298.64,57.78,140.18,0
2,3,15298.64,"-6540,0",8066.41,16825.05,16766.16,-58.89,114.81,0
3,4,16766.16,"-5950,0",2994.85,13811.01,13832.37,21.36,118.61,0
4,5,13832.37,"-6225,0",8046.40,15653.77,15600.11,-53.66,107.30,0
...,...,...,...,...,...,...,...,...,...
995,996,3.77,"-0,0",0.00,3.77,5.54,1.77,-2.21,1
996,997,5.54,"-0,0",0.00,5.54,0.83,-4.71,-9.32,1
997,998,0.83,"-0,0",0.00,0.83,13.25,12.42,1.59,1
998,999,13.25,"-0,0",0.00,13.25,10.31,-2.94,6.54,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22883.35,"-6050,0",5453.86,22287.21,22230.52,-56.69,249.29,0
1,2,22230.52,"-6275,0",3883.29,19838.81,19981.10,142.29,345.12,0
2,3,19981.10,"-6150,0",7525.49,21356.59,21346.06,-10.53,273.17,0
3,4,21346.06,"-6130,0",6948.29,22164.35,22227.30,62.95,280.51,0
4,5,22227.30,"-5815,0",5639.50,22051.80,22112.87,61.07,286.84,0
...,...,...,...,...,...,...,...,...,...
995,996,1.47,"-0,0",0.00,1.47,15.25,13.78,10.81,1
996,997,15.25,"-0,0",0.00,15.25,6.77,-8.48,-2.84,1
997,998,6.77,"-0,0",0.00,6.77,1.04,-5.73,-11.61,1
998,999,1.04,"-0,0",0.00,1.04,22.11,21.07,14.93,1


### Conjunto de datos de prueba

In [30]:
for case in data_dict:
    display(data_dict[case]['test_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,14217.87,-6025,8078.98,16271.85,16202.83,-69.02,29.76,0
1,2,16202.83,-5915,7993.50,18281.33,18287.13,5.80,103.63,0
2,3,18287.13,-6005,5997.67,18279.80,18294.11,14.31,27.16,0
3,4,18294.11,-6085,3013.11,15222.22,15202.29,-19.93,93.76,0
4,5,15202.29,-5800,7941.86,17344.15,17431.85,87.70,159.23,0
...,...,...,...,...,...,...,...,...,...
995,996,15406.65,-6030,4978.48,14355.13,14337.52,-17.61,61.26,1
996,997,14337.52,-6000,5721.51,14059.03,14009.35,-49.68,-44.29,1
997,998,14009.35,-6275,6313.19,14047.54,14181.99,134.45,161.32,1
998,999,14181.99,-6150,5914.01,13946.00,14007.40,61.40,181.88,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,17135.63,"-5975,0",7990.02,19150.65,19183.49,32.84,106.49,0
1,2,19183.49,"-6310,0",7021.06,19894.55,19852.62,-41.93,88.07,0
2,3,19852.62,"-5855,0",7068.29,21065.91,21015.88,-50.03,36.94,0
3,4,21015.88,"-5995,0",5991.07,21011.95,21025.10,13.15,-10.55,0
4,5,21025.10,"-6045,0",6914.83,21894.93,21927.46,32.53,-67.99,0
...,...,...,...,...,...,...,...,...,...
995,996,2535.76,"-6170,0",6732.66,3098.42,3084.18,-14.24,-244.63,1
996,997,3084.18,"-6050,0",5225.99,2260.17,2291.41,31.24,-67.43,1
997,998,2291.41,"-5735,0",7144.70,3701.11,3650.86,-50.25,-129.83,1
998,999,3650.86,"-5915,0",3212.68,948.54,914.60,-33.94,-248.70,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,23654.87,"-6005,0",4298.93,21948.80,21972.61,23.81,163.31,0
1,2,21972.61,"-6095,0",4622.96,20500.57,20617.61,117.04,322.76,0
2,3,20617.61,"-6175,0",9067.79,23510.40,23442.51,-67.89,221.84,0
3,4,23442.51,"-5990,0",3410.32,20862.83,20890.70,27.87,233.22,0
4,5,20890.70,"-5915,0",6810.38,21786.08,21803.84,17.76,215.53,0
...,...,...,...,...,...,...,...,...,...
995,996,10.65,"-0,0",0.00,10.65,30.77,20.12,26.54,1
996,997,30.77,"-0,0",0.00,30.77,0.47,-30.30,-3.40,1
997,998,0.47,"-0,0",0.00,0.47,4.08,3.61,0.87,1
998,999,4.08,"-0,0",0.00,4.08,1.82,-2.26,-3.71,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,17445.18,"-5905,0",6012.94,17553.12,17533.44,-19.68,16.06,0
1,2,17533.44,"-6110,0",8506.70,19930.14,20021.08,90.94,102.33,0
2,3,20021.08,"-5930,0",4407.28,18498.36,18474.61,-23.75,162.19,0
3,4,18474.61,"-6080,0",8762.80,21157.41,21038.06,-119.35,-12.14,0
4,5,21038.06,"-6210,0",8428.75,23256.81,23192.92,-63.89,-100.60,0
...,...,...,...,...,...,...,...,...,...
995,996,8.86,"-0,0",0.00,8.86,9.40,0.54,3.90,1
996,997,9.40,"-0,0",0.00,9.40,12.79,3.39,2.60,1
997,998,12.79,"-0,0",0.00,12.79,14.69,1.90,-0.82,1
998,999,14.69,"-0,0",0.00,14.69,9.11,-5.58,-12.74,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,17548.09,"-6265,0",4958.31,16241.40,16245.87,4.47,160.94,0
1,2,16245.87,"-5725,0",6084.29,16605.16,16550.86,-54.30,43.48,0
2,3,16550.86,"-5880,0",6957.27,17628.13,17694.10,65.97,93.95,0
3,4,17694.10,"-6025,0",7912.37,19581.47,19651.53,70.06,229.90,0
4,5,19651.53,"-5880,0",5996.13,19767.66,19771.94,4.28,319.27,0
...,...,...,...,...,...,...,...,...,...
995,996,9.49,"-0,0",0.00,9.49,2.52,-6.97,-20.78,1
996,997,2.52,"-0,0",0.00,2.52,1.60,-0.92,-5.93,1
997,998,1.60,"-0,0",0.00,1.60,7.92,6.32,-9.84,1
998,999,7.92,"-0,0",0.00,7.92,1.37,-6.55,-12.18,1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,16942.12,"-5825,0",5491.42,16608.54,16673.87,65.33,187.29,0
1,2,16673.87,"-5825,0",6897.10,17745.97,17875.86,129.89,237.48,0
2,3,17875.86,"-5965,0",6950.25,18861.11,18894.66,33.55,253.19,0
3,4,18894.66,"-5970,0",7962.56,20887.22,20937.92,50.70,387.03,0
4,5,20937.92,"-6135,0",5924.27,20727.19,20818.82,91.63,502.42,0
...,...,...,...,...,...,...,...,...,...
995,996,19.68,"-0,0",0.00,19.68,8.25,-11.43,0.72,1
996,997,8.25,"-0,0",0.00,8.25,3.65,-4.60,-8.36,1
997,998,3.65,"-0,0",0.00,3.65,8.40,4.75,4.92,1
998,999,8.40,"-0,0",0.00,8.40,23.49,15.09,16.35,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,18109.75,"-5940,0",6999.66,19169.41,19140.47,-28.94,290.50,0
1,2,19140.47,"-6320,0",7014.09,19834.56,19825.75,-8.81,346.46,0
2,3,19825.75,"-6145,0",7026.85,20707.60,20677.99,-29.61,326.59,0
3,4,20677.99,"-5805,0",4783.79,19656.78,19702.96,46.18,340.89,0
4,5,19702.96,"-6195,0",6592.28,20100.24,20021.69,-78.55,117.34,0
...,...,...,...,...,...,...,...,...,...
995,996,3.39,"-0,0",0.00,3.39,8.94,5.55,-9.07,1
996,997,8.94,"-0,0",0.00,8.94,13.46,4.52,1.87,1
997,998,13.46,"-0,0",0.00,13.46,5.20,-8.26,4.92,1
998,999,5.20,"-0,0",0.00,5.20,13.18,7.98,-1.52,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22465.51,"-6080,0",7789.55,24175.06,24208.87,33.81,-175.32,0
1,2,24208.87,"-6045,0",4948.93,23112.80,23139.14,26.34,-178.39,0
2,3,23139.14,"-5935,0",7036.17,24240.31,24285.57,45.26,-129.43,0
3,4,24285.57,"-5950,0",4812.44,23148.01,23215.86,67.85,74.70,0
4,5,23215.86,"-6310,0",5513.33,22419.19,22370.85,-48.34,67.20,0
...,...,...,...,...,...,...,...,...,...
995,996,13.88,"-0,0",0.00,13.88,5.95,-7.93,1.78,1
996,997,5.95,"-0,0",0.00,5.95,12.10,6.15,10.02,1
997,998,12.10,"-0,0",0.00,12.10,5.06,-7.04,-9.98,1
998,999,5.06,"-0,0",0.00,5.06,4.17,-0.89,-3.22,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,17698.38,"-5710,0",4618.64,16607.02,16580.51,-26.51,35.66,0
1,2,16580.51,"-5875,0",6004.30,16709.81,16702.31,-7.50,58.61,0
2,3,16702.31,"-6250,0",8001.28,18453.59,18456.14,2.55,-9.07,0
3,4,18456.14,"-5870,0",6931.83,19517.97,19595.08,77.11,-16.24,0
4,5,19595.08,"-6330,0",7449.42,20714.50,20644.03,-70.47,-108.89,0
...,...,...,...,...,...,...,...,...,...
995,996,1.11,"-0,0",0.00,1.11,1.53,0.42,-12.95,1
996,997,1.53,"-0,0",0.00,1.53,5.18,3.65,0.97,1
997,998,5.18,"-0,0",0.00,5.18,9.61,4.43,9.40,1
998,999,9.61,"-0,0",0.00,9.61,13.39,3.78,3.06,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22379.58,"-5785,0",5690.39,22284.97,22319.50,34.53,144.59,0
1,2,22319.50,"-5980,0",6499.32,22838.82,22799.39,-39.43,171.58,0
2,3,22799.39,"-6000,0",6305.86,23105.25,23145.46,40.21,213.48,0
3,4,23145.46,"-6095,0",5814.71,22865.17,22845.93,-19.24,195.21,0
4,5,22845.93,"-6145,0",6270.76,22971.69,22978.10,6.41,290.05,0
...,...,...,...,...,...,...,...,...,...
995,996,2.71,"-0,0",0.00,2.71,2.61,-0.10,-0.48,1
996,997,2.61,"-0,0",0.00,2.61,5.93,3.32,1.85,1
997,998,5.93,"-0,0",0.00,5.93,13.80,7.87,5.27,1
998,999,13.80,"-0,0",0.00,13.80,6.10,-7.70,-0.73,1


<!-- TODO -->

## Selección de características

In [31]:
for case in data_dict:
    for dataset in ['train_dataframe', 'test_dataframe']:
        data_dict[case][dataset] = data_dict[case][dataset].filter(items=['Variacion', 'Fugando combustible'])

### Conjunto de datos de entrenamiento

In [32]:
for case in data_dict:
    display(data_dict[case]['train_dataframe'])

,Variacion,Fugando combustible
0,40.80,0
1,-73.49,0
2,51.75,0
3,3.16,0
4,0.65,0
...,...,...
995,-43.59,1
996,-17.51,1
997,-37.18,1
998,-3.28,1


,Variacion,Fugando combustible
0,-42.81,0
1,-55.06,0
2,66.41,0
3,-59.69,0
4,36.40,0
...,...,...
995,-73.34,1
996,15.35,1
997,7.42,1
998,47.58,1


,Variacion,Fugando combustible
0,17.88,0
1,-12.33,0
2,54.59,0
3,20.41,0
4,44.61,0
...,...,...
995,12.19,1
996,-9.69,1
997,-4.40,1
998,10.76,1


,Variacion,Fugando combustible
0,-18.83,0
1,-1.26,0
2,41.52,0
3,8.68,0
4,34.63,0
...,...,...
995,1.46,1
996,-1.75,1
997,15.60,1
998,-13.96,1


,Variacion,Fugando combustible
0,58.02,0
1,121.04,0
2,58.73,0
3,22.45,0
4,32.78,0
...,...,...
995,20.11,1
996,-8.35,1
997,-4.60,1
998,-7.36,1


...

,Variacion,Fugando combustible
0,-61.69,0
1,-10.31,0
2,72.24,0
3,-36.77,0
4,10.39,0
...,...,...
995,1.14,1
996,-3.16,1
997,0.41,1
998,1.01,1


,Variacion,Fugando combustible
0,23.54,0
1,-45.10,0
2,80.80,0
3,33.26,0
4,61.75,0
...,...,...
995,-5.05,1
996,5.32,1
997,8.35,1
998,-9.41,1


,Variacion,Fugando combustible
0,37.72,0
1,-41.49,0
2,63.81,0
3,43.61,0
4,48.90,0
...,...,...
995,-2.60,1
996,0.74,1
997,2.01,1
998,-3.66,1


,Variacion,Fugando combustible
0,54.77,0
1,57.78,0
2,-58.89,0
3,21.36,0
4,-53.66,0
...,...,...
995,1.77,1
996,-4.71,1
997,12.42,1
998,-2.94,1


,Variacion,Fugando combustible
0,-56.69,0
1,142.29,0
2,-10.53,0
3,62.95,0
4,61.07,0
...,...,...
995,13.78,1
996,-8.48,1
997,-5.73,1
998,21.07,1


### Conjunto de datos de prueba

In [33]:
for case in data_dict:
    display(data_dict[case]['test_dataframe'])

,Variacion,Fugando combustible
0,-69.02,0
1,5.80,0
2,14.31,0
3,-19.93,0
4,87.70,0
...,...,...
995,-17.61,1
996,-49.68,1
997,134.45,1
998,61.40,1


,Variacion,Fugando combustible
0,32.84,0
1,-41.93,0
2,-50.03,0
3,13.15,0
4,32.53,0
...,...,...
995,-14.24,1
996,31.24,1
997,-50.25,1
998,-33.94,1


,Variacion,Fugando combustible
0,23.81,0
1,117.04,0
2,-67.89,0
3,27.87,0
4,17.76,0
...,...,...
995,20.12,1
996,-30.30,1
997,3.61,1
998,-2.26,1


,Variacion,Fugando combustible
0,-19.68,0
1,90.94,0
2,-23.75,0
3,-119.35,0
4,-63.89,0
...,...,...
995,0.54,1
996,3.39,1
997,1.90,1
998,-5.58,1


,Variacion,Fugando combustible
0,4.47,0
1,-54.30,0
2,65.97,0
3,70.06,0
4,4.28,0
...,...,...
995,-6.97,1
996,-0.92,1
997,6.32,1
998,-6.55,1


...

,Variacion,Fugando combustible
0,65.33,0
1,129.89,0
2,33.55,0
3,50.70,0
4,91.63,0
...,...,...
995,-11.43,1
996,-4.60,1
997,4.75,1
998,15.09,1


,Variacion,Fugando combustible
0,-28.94,0
1,-8.81,0
2,-29.61,0
3,46.18,0
4,-78.55,0
...,...,...
995,5.55,1
996,4.52,1
997,-8.26,1
998,7.98,1


,Variacion,Fugando combustible
0,33.81,0
1,26.34,0
2,45.26,0
3,67.85,0
4,-48.34,0
...,...,...
995,-7.93,1
996,6.15,1
997,-7.04,1
998,-0.89,1


,Variacion,Fugando combustible
0,-26.51,0
1,-7.50,0
2,2.55,0
3,77.11,0
4,-70.47,0
...,...,...
995,0.42,1
996,3.65,1
997,4.43,1
998,3.78,1


,Variacion,Fugando combustible
0,34.53,0
1,-39.43,0
2,40.21,0
3,-19.24,0
4,6.41,0
...,...,...
995,-0.10,1
996,3.32,1
997,7.87,1
998,-7.70,1


<!-- TODO -->

## Unificación de conjunto de datos

In [34]:
complete_train_data = pd.concat([element['train_dataframe'] for element in data_dict.values()])

train_data = (
    complete_train_data.drop(columns='Fugando combustible'), 
    complete_train_data['Fugando combustible']
)

In [35]:
segmented_test_data = [
    (element['leak_value'], element['test_dataframe']) for element in data_dict.values()
]

test_data = [(
    'general',
    pd.concat([element['test_dataframe'] for element in data_dict.values()], ignore_index=True)
)]

## Implementación de modelos

### Definición de métricas

In [36]:
def leak_detection_delay(y_true: np.array, y_pred: np.array) -> int:
    real_first_leak_day = 0
    while y_true[real_first_leak_day] != 1: real_first_leak_day += 1

    pred_first_leak_day = real_first_leak_day
    while y_pred[pred_first_leak_day] != 1: pred_first_leak_day += 1

    return (pred_first_leak_day - real_first_leak_day)

In [37]:
def performance_metrics(y_true, y_pred, leak_value = 0.013) -> dict:
    return {
        # Metricas clasicas de ML
            'accuracy_score':   accuracy_score(y_true, y_pred),
            'f1_score':         f1_score(y_true, y_pred),
            'recall_score':     recall_score(y_true, y_pred),
            'precision_score':  precision_score(y_true, y_pred),
            'confusion_matrix': confusion_matrix(y_true, y_pred),
        # Metricas propias del problema
            'false_positives':  confusion_matrix(y_true, y_pred).ravel()[1],
            'leak_delay':       leak_detection_delay(y_true, y_pred),
            'leak_loss':        leak_detection_delay(y_true, y_pred) * leak_value * 60 * 24
    }

### Modelos clásicos

In [38]:
from sklearn.model_selection import GridSearchCV
import pickle as pkl

param_grids_path = '../../pkl/param_grids'

def train_classic_model(classifier_class, param_grid, train_data, cv = 5):
    param_pkl_path = f'{param_grids_path}/{classifier_class.__name__}.pkl'
    classifier = None
    X_train, y_train = train_data
    if os.path.exists(param_pkl_path):  # Param grid already known
        param_grid = pkl.load(open(param_pkl_path, 'rb'))
        classifier = classifier_class(**param_grid)
        classifier.fit(X_train, y_train)
    else:
        grid_search = GridSearchCV(
            estimator = classifier_class(),
            param_grid = param_grid,
            cv = cv
        )
        grid_search.fit(X_train, y_train)
        classifier = grid_search.best_estimator_
        pkl.dump(
            obj = grid_search.best_params_,
            file = open(param_pkl_path, 'wb')
        )
    return classifier

In [39]:
def test_classic_model(classifier, test_list) -> list[dict]:
    results = list()
    for (leak_value, test_data) in test_list:
        X_test = test_data.drop(columns='Fugando combustible')
        y_test = test_data['Fugando combustible']
        y_pred = classifier.predict(X_test)
        results.append((
            leak_value,
            performance_metrics(y_test, y_pred, leak_value)
        ))
    return results

##### Random Forest Classifier

In [40]:
from sklearn.ensemble import RandomForestClassifier

rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_model = train_classic_model(RandomForestClassifier, rf_param_grid, train_data)

rf_seg_results = test_classic_model(rf_model, segmented_test_data)
rf_results = test_classic_model(rf_model, test_data)

##### Naive Bayes Classifier

In [41]:
from sklearn.naive_bayes import GaussianNB

gnb_param_grid = {
    'var_smoothing': [1e-11, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

gnb_model = train_classic_model(GaussianNB, gnb_param_grid, train_data)

gnb_seg_results = test_classic_model(gnb_model, segmented_test_data)
gnb_results = test_classic_model(gnb_model, test_data)

##### KNN Classifier

In [42]:
from sklearn.neighbors import KNeighborsClassifier

knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

knn_model = train_classic_model(KNeighborsClassifier, knn_param_grid, train_data)

knn_seg_results = test_classic_model(knn_model, segmented_test_data)
knn_results = test_classic_model(knn_model, test_data)

##### XGBoost Classifier

In [43]:
from xgboost import XGBClassifier

xgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

xgb_model = train_classic_model(XGBClassifier, xgb_param_grid, train_data)

xgb_seg_results = test_classic_model(xgb_model, segmented_test_data)
xgb_results = test_classic_model(xgb_model, test_data)

##### _Ensemble Classifier_

In [44]:
from sklearn.ensemble import VotingClassifier

clf_model = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier()),
        ('nb', GaussianNB()),
        ('knn', KNeighborsClassifier()),
        ('XGB', XGBClassifier())
    ],
    voting='hard'
)

clf_model.fit(train_data[0], train_data[1])

clf_seg_results = test_classic_model(clf_model, segmented_test_data)
clf_results = test_classic_model(clf_model, test_data)

### Modelos _Deep Learning_

In [45]:
def test_dl_model(model, test_list):
    result_list = []
    for leak_value, test_dataframe in test_list:
        X_test = test_dataframe.drop(columns='Fugando combustible')
        y_test = test_dataframe['Fugando combustible']
        
        y_pred = (model.predict(X_test, verbose=0) > 0.5).astype(int)

        result_list.append(performance_metrics(y_test, y_pred, leak_value))

    return result_list

#### Único modelo de _Deep Learning_

In [46]:
X_train, y_train = train_data

##### Modelo unicapa

In [47]:
unil_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
    tf.keras.layers.LSTM(15, dropout=0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

unil_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = unil_model.fit(
    x = X_train,
    y = y_train,
    epochs = 50,
    batch_size = 8192,
    verbose = 0
)

dl_unil_seg_results = test_dl_model(unil_model, segmented_test_data)
dl_unil_results = test_dl_model(unil_model, test_data)

##### Modelo multicapa

In [48]:
multil_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
    tf.keras.layers.LSTM(25, return_sequences=True, dropout=0.2),
    tf.keras.layers.LSTM(40, return_sequences=True, dropout=0.2),
    tf.keras.layers.LSTM(60, dropout=0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

multil_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = multil_model.fit(
    x = X_train,
    y = y_train,
    epochs = 50,
    batch_size = 8192,
    verbose = 0
)

dl_multil_seg_results = test_dl_model(multil_model, segmented_test_data)
dl_multil_results = test_dl_model(multil_model, test_data)

#### Múltiples modelos de _Deep Learning_

In [49]:
dl_multim_seg_results = []
for case in data_dict:
    X_train = data_dict[case]['train_dataframe'].drop(columns='Fugando combustible')
    y_train = data_dict[case]['train_dataframe']['Fugando combustible']
    
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
        tf.keras.layers.LSTM(15, dropout=0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        x = X_train,
        y = y_train,
        epochs = 50,
        batch_size = 8192,
        verbose = 0
    )

    leak_value = data_dict[case]['leak_value']

    X_test = data_dict[case]['test_dataframe'].drop(columns='Fugando combustible')
    y_test = data_dict[case]['test_dataframe']['Fugando combustible']
    
    y_pred = (model.predict(X_test, verbose=0) > 0.5).astype(int)

    dl_multim_seg_results.append(performance_metrics(y_test, y_pred, leak_value))
    # model.save(f'../../pkl/models/{case}.keras')

In [50]:
dl_multim_results = []
for case in data_dict:
    X_train = data_dict[case]['train_dataframe'].drop(columns='Fugando combustible')
    y_train = data_dict[case]['train_dataframe']['Fugando combustible']
    
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
        tf.keras.layers.LSTM(15, dropout=0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        x = X_train,
        y = y_train,
        epochs = 50,
        batch_size = 8192,
        verbose = 0
    )

    leak_value = test_data[0][0]

    X_test = test_data[0][1].drop(columns='Fugando combustible')
    y_test = test_data[0][1]['Fugando combustible']
    
    y_pred = (model.predict(X_test, verbose=0) > 0.5).astype(int)

    dl_multim_seg_results.append(performance_metrics(y_test, y_pred, leak_value))
    # model.save(f'../../pkl/models/{case}.keras')

#### Modelo _ensemble_ de mini\_modelos de _Deep Learning_

In [ ]:
class HomebrewEnsembleClassifier:
    def __init__(self, model_list) -> None:
        self.model_list = model_list

    
    def predict(X_test) -> np.ndarray:
        # TODO
        raise NotImplementedError()

## Visualización de resultados

In [ ]:
def display_performance_table(results: list[dict]) -> None:
    display(pd.DataFrame.from_records(results).drop(columns='confusion_matrix'))